In [1]:
import os, glob
import matplotlib.pyplot as plt

In [2]:
### Modifiable Parameters ###
# Number of lattice
nx_site = 70
ny_site = 70
# Binary cutoff
cutoff_1 = 3000
cutoff_2 = 10000

In [3]:
### Static ###
from astropy.io import fits
import scipy.ndimage
import numpy as np
from scipy import signal
from scipy.optimize import curve_fit
import os

# Global parameters
### Global Parameters ###
# Image dimensions
img_size_x = 512 # The image dimensions from Andor (512x512).
img_size_y = 512 # The image dimensions from Andor (512x512).
# Background image LPF parameters
bkg_frame_id  = 4 # Frame index of the background frame
dat_frame_ids = [1, 2] # Frame index of the data frame
smear_length = 256.0/50.0 # length scale of blur in pixels
q0 = np.pi / smear_length
n = 2.
# Oversize image
resample_rate = 3 # Expand ratio along one axis. So 1 pixel in raw image corresponds to 9 pixels now. 
# Image Chop (in resized image index)
decon_MinX = 350
decon_MaxX = 1250
decon_MinY = 350
decon_MaxY = 1250
# Point spread function
PSF_wx = 4.26187
PSF_wy = 4.26187
# Image rotation angle
rot_ang = 17.084 # degree clockwise
# Phase detection
rebin_rpxl = 3      # Re-binning number in unit of resampled pixels.
xlat_pxl   = 3.0508 # X lattice spacing in rebinned pixels.
ylat_pxl   = 3.0494 # Y lattice spacing in rebinned pixels.
dtheta_deg = 0.414  # Angle difference due to X-Y non-orthogonality.
# Final chopping for phase finding
x0 = 600
y0 = 600

In [4]:
data_file = os.path.join(os.getcwd(), "2020/08/15/Archive/Data", "12-07-2018_15_33_01.fits")
result = {}

In [ ]:
### Analysis Script ###
imgs_dig = []
imgs_flt = []

####################
### Import image ###
####################
fname = data_file
print(fname)
print("Loading new file. (\"{}\")".format(os.path.split(fname)[-1]))
imgs = fits.getdata(fname).astype(int)

###################################
### Background image processing ###
###################################
# Build LPF matrix
print("Processing background image!")
qxs = np.linspace(-np.pi, np.pi, img_size_x, endpoint=False)
qys = np.linspace(-np.pi, np.pi, img_size_y, endpoint=False)
qxv, qyv = np.meshgrid(qxs, qys)
bkg_lpf = 1.0/(1.0+(np.sqrt(qxv**2+qyv**2)/q0)**(2*n)) # Filter mask for the shifted Fourier transform
# Apply LPF to the FFT image
mask = np.tile([[1, -1],[-1, 1]], [int(img_size_x/2), int(img_size_y/2)]) # Mask for shifting the origin of FFT
img_bkg_raw = imgs[bkg_frame_id] # Get the backgound frame
img_bkg_fft = np.fft.fft2(img_bkg_raw*mask) # Fourier transfrom the image
img_bkg_lpf = np.real(np.fft.ifft2(img_bkg_fft*bkg_lpf)*mask) # Apply filter and Inverse FFT

#############################
### Dark field statistics ###
#############################
# Fitting function for dark counts fluctuation
def Gaussian(x, xc, w, amp):
    return 1.*amp/(w*np.sqrt(2*np.pi))*np.exp(-1./2.*((x-xc)/w)**2)
# Prosses the dark field image: background subtraction, resizing, build histogram
img_bkg = img_bkg_raw-img_bkg_lpf # Subtract the dark field image with LPF image. 
img_bkg = scipy.ndimage.zoom(img_bkg, resample_rate, order=0) # Resize the background image
img_bkg = img_bkg[decon_MinX:decon_MaxX, decon_MinY:decon_MaxY] # Chop image
hist, bin_edges = np.histogram(img_bkg, bins=100, range=(-100, 0)) # Build intesnsity histogram
# Fit the negative histogram with a Gaussian
popt, pcov = curve_fit(Gaussian, bin_edges[:-1], hist, p0=[0., 28., hist[-1]]) # Fit the Gaussian
cutoff = 2*np.sqrt(2)*popt[1] # Cutoff guess for finding phase
print("Background intensity cutoff: {:.2f}".format(cutoff))

#############################
### Data image processing ###
#############################
# Loop over data images
for ii, _img in enumerate(imgs[dat_frame_ids]):
    # Background subtraction
    _img_bkdsub = _img-img_bkg_lpf
    # Image oversize
    _img_os = scipy.ndimage.zoom(_img_bkdsub, resample_rate, order=0)
    # Chop the image
    _img_chop = _img_os[decon_MinX:decon_MaxX, decon_MinY:decon_MaxY]
    
    #############################
    ### Wienner deconvolution ###
    #############################
    # Image size
    x_size, y_size = _img_chop.shape
    # Point-Spread function
    def Gaussian2D(x, y, xc, yc, wx, wy):
        return Gaussian(x, xc, wx, 1)*Gaussian(y, yc, wy, 1)
    xx = np.linspace(0, x_size, x_size, endpoint=False)
    yy = np.linspace(0, y_size, y_size, endpoint=False)
    xv, yv = np.meshgrid(xx, yy)
    PSF = Gaussian2D(xv, yv, x_size/2, y_size/2, PSF_wx, PSF_wy) \
          + (15.6794/PSF_wx)**2*(175.56/1616.4)*Gaussian2D(xv, yv, x_size/2, x_size/2, 15.6794, 15.6794)

    # Fourier transformation of PSF (linear response function)
    mask = np.tile([[1,-1],[-1,1]], (int(x_size/2), int(y_size/2))) # TODO: MAKE SURE THIS IS INTEGER
    PSF_fft = np.fft.fft2(PSF*mask)*mask # FFT of point spread function
    H   = PSF_fft
    HCC = np.conj(H)
    
    # Fermi-Dirac filter
    def FermiDirac(x, y, q0, qw):
        return 1./(np.exp((np.sqrt(x**2+y**2)-q0)/qw)+1.)
    def FermiDiracFilter(img_size, q0, qw):
        xx = np.linspace(-np.pi, np.pi, img_size[0], endpoint=False)
        yy = np.linspace(-np.pi, np.pi, img_size[1], endpoint=False)
        xv, yv = np.meshgrid(xx, yy)
        return FermiDirac(xv, yv, q0, qw)
    
#     q0 = np.array([2*np.pi/3.0, 2*np.pi/3.0, 2*np.pi/3.0, 2*np.pi/3.0, 2*np.pi/3.0, 2*np.pi/3.0, 2*np.pi/3.0, np.pi/3.0])
#     qw = np.array([2*np.pi/30.0, 2*np.pi/30.0, 2*np.pi/30.0, 2*np.pi/30.0, 2*np.pi/30.0, 2*np.pi/30.0, 2*np.pi/30.0, 2*np.pi/30.0])
    q0 = 2.*np.pi/3.0
    qw = 2.*np.pi/30.0
    resid = 0.01 # TODO: GLOBAL PARAMETERS
    
    _img_chop[_img_chop<cutoff] = 0. # Intensity cutoff
    _img_fft = np.fft.fft2(_img_chop*mask) # FFT of image
    _img_fdf = _img_fft*FermiDiracFilter((x_size, y_size), q0, qw) # Apply Fermi-Dirac filter
    _img_dec = _img_fdf*HCC/(HCC*H+resid) # Wiener deconvolution
    _img_dec = np.real(np.fft.ifft2(_img_dec)*mask) # Inverse FFT
    
    ######################
    ### Image rotation ###
    ######################
    _img_rot = scipy.ndimage.rotate(_img_dec, rot_ang)
    
    ##################
    ### Find phase ###
    ##################
    print("Finding phase!")
    # Helper functions
    rad_per_deg = np.pi/180.
    # pixels per real lattice site
    x_pixels_per_lattice = rebin_rpxl*xlat_pxl
    y_pixels_per_lattice = rebin_rpxl*ylat_pxl
    # Function for getting phase
    def bin_sites(img, x_offset, y_offset):
        x_site, y_site = np.meshgrid(np.arange(int(-nx_site/2), int(-nx_site/2+nx_site)), 
                                     np.arange(int(-ny_site/2), int(-ny_site/2+ny_site)))
        # TODO: Probably a cosine(delta_theta) factor is missing but it must be very close to 1
        x_pxl = np.round(x0+x_site*x_pixels_per_lattice+x_offset).astype(int)
        # Correction for non-orthogonality between the two lattice axes
        y_pxl = np.round(y0+y_site*y_pixels_per_lattice+y_offset
                         +x_site*x_pixels_per_lattice*np.sin(dtheta_deg*rad_per_deg)).astype(int) # Correction for non-orthogonality
        # TODO: Not the best way of binning but it's ok for now probably
        site_intensity = np.zeros((int(nx_site-1), int(ny_site-1)))
        for i in range(int(nx_site)-1):
            for j in range(int(ny_site)-1):
                # TODO: maybe try taking the mean too
                _x0 = x_pxl[i,j]; _x1 = x_pxl[i+1,j+1];
                _y0 = y_pxl[i,j]; _y1 = y_pxl[i+1,j+1];
                site_intensity[i,j] = img[_x0:_x1, _y0:_y1].sum()
        return site_intensity
    def cos(x, x_offset, x_pxl_per_site):
        return np.cos(2*np.pi/x_pxl_per_site*(x-x_offset))
    def fit_cos(averaged_intensity):
        xdat = np.linspace(0, 8, 9)
        ydat = (averaged_intensity-np.average(averaged_intensity))/(.5*np.ptp(averaged_intensity))
        popt, pcov = curve_fit(cos, xdat, ydat, p0=[np.asarray(ydat == np.max(ydat)).nonzero()[0][0], 9.0])
        return popt[0]
    def get_counts(img, x_offset, y_offset):
            img = bin_sites(img, x_offset, y_offset)
            img[img<100.*cutoff] = 0
            return img.sum()
    def get_phase(img):
        intensity_counts = [[get_counts(img, ii, jj) for jj in range(9)] for ii in range(9)]
        x_phase = fit_cos(np.average(intensity_counts, axis=1))
        y_phase = fit_cos(np.average(intensity_counts, axis=0))
        return x_phase, y_phase
    
    # Get phase
    _x_phase, _y_phase = get_phase(_img_rot)
    print("X phase [pixel]: {:.3f}".format(_x_phase))
    print("Y phase [pixel]: {:.3f}".format(_y_phase))
    
    ######################
    ### Binarize image ###
    ######################
    print("Binarizing image!")
    _img_flt = bin_sites(_img_rot, _x_phase, _y_phase)
    _img_bin = np.copy(_img_flt)
    _img_bin[_img_bin<cutoff_1] = 0
    _img_bin[(_img_bin>cutoff_1) & (_img_bin<cutoff_2)] = 1
    _img_bin[_img_bin>cutoff_2] = 2
    
    ####################
    ### Save results ###
    ####################
    if ii == 0:
        img_raw = _img_bkdsub
        n = np.sum(_img_bin)
        x_phase = _x_phase
        y_phase = _y_phase
        
        result['n'] = n
        result['X phase'] = x_phase
        result['Y phase'] = y_phase
    
    imgs_flt.append(_img_flt)
    imgs_dig.append(_img_bin)

In [ ]:
### Plots ###
### Raw Image ###
ax.imshow(img_raw)
ax.axis('off')

In [ ]:
### Plots ###
### Binary Image ###
ax.imshow(imgs_dig[0])
ax.axis('off')

In [ ]:
### Plots ###
### Histogram ###
*_, = ax.hist(imgs_flt[0].ravel(), 50)

In [ ]:
### Plots ###
### Phase ###
x_phase = df['X phase'].values
y_phase = df['Y phase'].values
ax.plot(x_phase)
ax.plot(y_phase)